In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

win_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'win_rate')
pick_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'pick_rate')
power_level  = ctrl.Consequent(np.arange(1, 6, 1), 'power_level')

win_rate["L"] = fuzz.trapmf(win_rate.universe, [0, 0, 40, 45])
win_rate["ML"] = fuzz.trimf(win_rate.universe, [40, 45, 50])
win_rate["M"] = fuzz.trimf(win_rate.universe, [45, 50, 55])
win_rate["MH"] = fuzz.trimf(win_rate.universe, [50, 55, 60])
win_rate["H"] = fuzz.trapmf(win_rate.universe, [55, 60, 100, 100]) 

pick_rate["L"] = fuzz.trimf(pick_rate.universe, [0, 0, 25])
pick_rate["ML"] = fuzz.trimf(pick_rate.universe, [0, 25, 50])
pick_rate["M"] = fuzz.trimf(pick_rate.universe, [25, 50, 75])
pick_rate["MH"] = fuzz.trimf(pick_rate.universe, [50, 75, 100])
pick_rate["H"] = fuzz.trimf(pick_rate.universe, [75, 100, 100])

power_level["L"] = fuzz.trimf(power_level.universe, [1, 1, 2])
power_level["ML"] = fuzz.trimf(power_level.universe, [1, 2, 3])
power_level["M"] = fuzz.trimf(power_level.universe, [2, 3, 4])
power_level["MH"] = fuzz.trimf(power_level.universe, [3, 4, 5])
power_level["H"] = fuzz.trimf(power_level.universe, [4, 5, 5])

In [ ]:
win_rate.view()
pick_rate.view()
power_level.view()

In [ ]:
rule1 = ctrl.Rule(win_rate['L'] & pick_rate['L'], power_level['L'])
rule2 = ctrl.Rule(win_rate['L'] & pick_rate['ML'], power_level['L'])
rule3 = ctrl.Rule(win_rate['L'] & pick_rate['M'], power_level['L'])
rule4 = ctrl.Rule(win_rate['L'] & pick_rate['MH'], power_level['L'])
rule5 = ctrl.Rule(win_rate['L'] & pick_rate['H'], power_level['L'])

rule6 = ctrl.Rule(win_rate['ML'] & pick_rate['L'], power_level['L'])
rule7 = ctrl.Rule(win_rate['ML'] & pick_rate['ML'], power_level['ML'])
rule8 = ctrl.Rule(win_rate['ML'] & pick_rate['M'], power_level['ML'])
rule9 = ctrl.Rule(win_rate['ML'] & pick_rate['MH'], power_level['M'])
rule10 = ctrl.Rule(win_rate['ML'] & pick_rate['H'], power_level['M'])

rule11 = ctrl.Rule(win_rate['M'] & pick_rate['L'], power_level['ML'])
rule12 = ctrl.Rule(win_rate['M'] & pick_rate['ML'], power_level['ML'])
rule13 = ctrl.Rule(win_rate['M'] & pick_rate['M'], power_level['M'])
rule14 = ctrl.Rule(win_rate['M'] & pick_rate['MH'], power_level['M'])
rule15 = ctrl.Rule(win_rate['M'] & pick_rate['H'], power_level['MH'])

rule16 = ctrl.Rule(win_rate['MH'] & pick_rate['L'], power_level['M'])
rule17 = ctrl.Rule(win_rate['MH'] & pick_rate['ML'], power_level['M'])
rule18 = ctrl.Rule(win_rate['MH'] & pick_rate['M'], power_level['MH'])
rule19 = ctrl.Rule(win_rate['MH'] & pick_rate['MH'], power_level['MH'])
rule20 = ctrl.Rule(win_rate['MH'] & pick_rate['H'], power_level['H'])

rule21 = ctrl.Rule(win_rate['H'] & pick_rate['L'], power_level['ML'])
rule22 = ctrl.Rule(win_rate['H'] & pick_rate['ML'], power_level['M'])
rule23 = ctrl.Rule(win_rate['H'] & pick_rate['M'], power_level['MH'])
rule24 = ctrl.Rule(win_rate['H'] & pick_rate['MH'], power_level['H'])
rule25 = ctrl.Rule(win_rate['H'] & pick_rate['H'], power_level['H'])

In [ ]:
power_level_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, 
                                       rule6, rule7, rule8, rule9, rule10,
                                       rule11, rule12, rule13, rule14, rule15, 
                                       rule16, rule17, rule18, rule19, rule20, 
                                       rule21, rule22, rule23, rule24, rule25])

power_level_sim = ctrl.ControlSystemSimulation(power_level_ctrl)

In [ ]:
power_level_sim.input['win_rate'] = 55.0
power_level_sim.input['pick_rate'] = 35.0

power_level_sim.compute()

In [ ]:
print(power_level_sim.output['power_level'])
power_level.view(sim=power_level_sim)

In [1]:
import pandas as pd

csv_files = [
    'CardData/AFR.csv', 'CardData/AKR.csv', 'CardData/BRO.csv', 'CardData/CORE.csv', 'CardData/DBL.csv', 
    'CardData/DMU.csv', 'CardData/DOM.csv', 'CardData/ELD.csv', 'CardData/HGB.csv', 'CardData/IKO.csv',
    'CardData/KHM.csv', 'CardData/KLR.csv', 'CardData/KTK.csv', 'CardData/LCI.csv', 'CardData/LTR.csv', 
    'CardData/MAT.csv', 'CardData/MID.csv', 'CardData/MKM.csv', 'CardData/MOM.csv', 'CardData/NEO.csv',
    'CardData/ONE.csv', 'CardData/RAVM.csv', 'CardData/RIX.csv', 'CardData/SIR.csv', 'CardData/SNC.csv', 
    'CardData/STX.csv', 'CardData/THB.csv', 'CardData/VOW.csv', 'CardData/WOE.csv',
]

df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.to_csv('CardData/CombinedSets.csv', index=False)

In [3]:
import pandas as pd

df = pd.read_csv('CardData/CombinedSets.csv')

df.replace('null', pd.NA, inplace=True)

cleaned_df = df.dropna(subset=['GP WR', '% GP'], inplace=True)

cleaned_df = df.drop_duplicates(subset=['Name'], inplace=True)

df.to_csv('CardData/CleanedCombinedSets.csv', index=False)